# Introduzione ai Test Statistici

## Cos'è un test d'ipotesi

[I test **d'ipotesi**, prendono questo nome perchè vanno a verificare la bontà di una ipotesi.](https://en.wikipedia.org/wiki/Statistical_hypothesis_testing)
Per ipotesi è da intendersi un'affermazione che ha come oggetto accadimenti nel mondo reale, che si presta ad essere confermata o smentita dai dati osservati sperimentalmente.
L'ipotesi da verificare è detta **nulla** e si indica con $H_0$, $H_1$ invece è l'ipotesi **alternativa**.

<br>
Lo studio di serie storiche con modelli statistici (ARMA, ARIMA, SARIMA, Holt-Winters,...) presuppone che la distribuzione abbia alcune caratteristiche come:
* stazionarietà
* correlazione dei dati (i dati sono indipendentemente distribuiti?)
* distibuzione normale dei dati
<br>

Per verificare se sussistono queste caratteristiche bisogna effettuare alcuni test statistici. In python la principale libreria che racchiude questi strumenti è [statsmodels](https://www.statsmodels.org/stable/index.html), in particolare il modulo *tsa*.

## Cos'è un test di radice unitaria

I test di radice unitaria sono dei tipi di test che vengono usati per stabilire se la serie temporale in esame presenta una radice unitaria al suo interno, la quale permette di affermare se il processo generante i dati risulta essere *trend-stazionario* oppure *differenza-stazionario*.
* trend-stazionario: funzione aleatoria
* differenza-stazionario: funzione variante nel tempo in modo casuale

## Caricamento di Airpassengers

Airpassengers, una sequenza mensile del numero di passeggeri sui voli internazionali tra il 1949 e il 1960.
Anticipiamo che questa serie storica ha un trend lineare crescente e presenta un effetto stagionale moltiplicativo (ovvero ogni anno si ripetono più o meno le stesse fluttuazioni ma accentuandosi).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings("ignore")

Per prima cosa carichiamo la serie e vediamo:
* i primi 5 valori
* il plot dell'originale

In [ ]:
airpassengers = pd.read_csv("airpassenger.csv")
airpassengers.columns = ["Time","Passengers"]
airpassengers.head()

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(airpassengers.shape[0]), airpassengers["Passengers"].values, color='tab:red')
plt.gca().set(title="Airpassengers", xlabel="Time", ylabel="Passengers")
plt.show()

## Componenti della serie

Una serie storica ha solitamente quattro componenti:
* *trend*
* *ciclo*
* *stagionalità*
* *componente erratica*
<br>

In tsa esiste una funzione che decompone la serie e ci fa vedere 3 delle quattro componenti sopra elencate.
<br>

**N.B. La differenza tra ciclo e stagionalità. La stagionalità si presenta sui dodici mesi, il ciclo può essere mensile, seetimanale, ... non confondiamo.**

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
decompose = seasonal_decompose(airpassengers["Passengers"], model = "multiplicative", period = 12)

Sapevamo già che la serie è mensile, quindi frequenza 12, il model indica il tipo di stagionalità, se vediamo il grafico i picchi crescono al passare dei periodi, quindi l'interazione tra le componenti è moltiplicativa.
<br>
Nei grafici di seguito vediamo:
1. La serie originale
2. Il trend
3. La stagionalità
4. La componente erratica

In [ ]:
plt.figure(figsize=(16,8))
decompose.plot()
plt.show()

## Autocorrelazione

Data una serie temporale $X_{t}$ con $t:{1, 2, ..., N}$ ci dice se e in che misura l'elemento $X_t$ abbia una relazione con un istante di tempo precedente $X_{t-h}$. L'analisi dell'andamento della funzione di autocorrelazione al variare di questo h è di fondamentale importanza per valutare la cosiddetta "*memoria*" della serie storica.
In parole semplici, l'analisi delle autocorrelazioni ci dice quanto di ciò che vediamo all'istante $t$ risenta di ciò che è successo a $(t-h)$.

Lo strumento grafico per valutare se una serie è stazionaria o no è il **correlogramma**, un grafico a barre che riporta $\rho_k$ sull'asse delle ordinate e $k$ (ovvero i lag) sulle ascisse.
Affinchè non ci sia una forte relazione tra i periodi il (*non significativamente correlati*) le barre dovrebbero muoversi entro le bande con un andamento "variabile".

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf

### Autocorrelazioni globali

In [ ]:
acf = acf(airpassengers["Passengers"], nlags = 40,qstat= True, fft=False)

Ritorna una tupla di tre array:
* le autocorrelazioni
* la q stat di Box - Jenkins per ogni autocorrelazione
* il p-value associato ad ogni correlazione

In [ ]:
plt.figure(figsize=(16,8))
plot_acf(airpassengers["Passengers"], lags = 40)
plt.show()

Nel nostro caso si vede sia una dipendenza dal trend, le barre hanno un andamento lento e decrescente all'aumentare di $k$, sia una dipendenza stagionale data *dall'onda* formata da $\rho_k$ con i valori più alti ogni 12 periodi.

Possiamo concludere che le correlazioni associate sono significativamente diverse da zero.

### Autocorrelazioni parziali

Con le autocorrelazioni globali vediamo l'effetto su $X_t$ dovuto a $X_{t-h}$, ma come capire se c'è un effetto dovuto agli istanti intermedi? Ci vengono in supporto le **autocorrelazioni parziali** ([pacf](https://www.statsmodels.org/devel/generated/statsmodels.tsa.stattools.pacf.html)) che misurano la dipendenza tra i due istanti al netto dell'influenza di quelli intermedi.

In [ ]:
prt_acf = pacf(airpassengers["Passengers"])

In [ ]:
plt.figure(figsize=(16,8))
plot_pacf(airpassengers["Passengers"])
plt.show()

Anche con le autocorrelazioni parziali possiamo affermare quanto giù detto per le totali.

Prima di introdurre i due test creiamo una funzione per valutare se *accettare* o *rigettare* l'ipotesi nulla.
Per farlo và confrontato il [*p-value*](https://en.wikipedia.org/wiki/P-value) con il [livello di significatività del test](https://en.wikipedia.org/wiki/Statistical_significance) indicato con $\alpha$, solitamente il valore di quest'ultimo è pari a 0.05 in quanto si ritiene che sufficientemente piccolo da poter concludere che sia **"piuttosto improbabile"** che la differenza osservata sia dovuta al semplice caso. Altri valori utilizzati sono 0.1 in fase esplorativa di alcuni test e uno più stringente 0.01.

In [ ]:
# funzione che confronta il p-value con il livello di significatività del test
# e scrive se accettare o rifiutare l'ipotetsi nulla.
def hypothesis(p_value, alpha):
    if p_value<= alpha:
        print("Non accettiamo H0")
    else:
        print("Accettiamo H0")

### Augmented Dikey-Fuller test
Il test di [Dikey-Fuller](https://en.wikipedia.org/wiki/Dickey%E2%80%93Fuller_test), che in statsmodel presenta la sua versione [*Augmented*](https://www.statsmodels.org/devel/generated/statsmodels.tsa.stattools.adfuller.html?highlight=adfuller#statsmodels.tsa.stattools.adfuller), ha sotto l'ipotesi nulla:
* **H0**: presenza di una radice unitaria alla frequenza zero

In questo caso ho scelto di lasciare i valori di default ai parametri della funzione tranne che per:
* regression: nella regressione che applica il test introduciamo sia una costante che il nostro trend scegliendo il valore "*ct*"

In [ ]:
from statsmodels.tsa.stattools import adfuller
adf = adfuller(airpassengers["Passengers"], regression="ct")

In [ ]:
adf

In [ ]:
hypothesis(p_value=round(adf[1],2), alpha=0.05)

Il p-value pari a 0.54 ciò fa si che non possiamo rigettare l'ipotesi nulla, nel nostro caso la presenza di radici unitarie.

*N.B. I valori vicino a 1 sono detti* ***critici***

### Kwiatkowski, Phillips, Schmidt e Shin (KPSS) test

Il [test KPSS](https://en.wikipedia.org/wiki/KPSS_test) si utilizza quando si vuole confrontare l'ipotesi nulla di assenza di radici unitarie con l'ipotesi alternativa che la serie abbia una (o più) radici unitarie. Quindi le due ipotesi sono invertite rispetto al test precedente:
* $H_0$: NON presenta radice unitaria
* $H_1$: la serie presenta una o più radici unitarie

In [ ]:
from statsmodels.tsa.stattools import kpss
kpss_test = kpss(airpassengers["Passengers"], regression = 'ct', nlags = 13, )

**N.B.**
* Cosa è *nlags*? I **lag** sono le distanze tra le unità temporali, tra $X_{t0}$ e $X_{t3}$ ci sono 3 lag.
* Perchè proprio 13? La formula che usa statsmodel per scegliere i lag è $(12 * (n / 100) ^\frac{1}{4})$ dove $n$ sono le osservazioni.

In [ ]:
kpss_test

In [ ]:
hypothesis(round(kpss_test[1],2), alpha=0.05)

Anche questo test ci suggerisce che la serie **presenta** una radice unitaria.

### Rimozione del trend
Visto il trend lineare, dovrebbe essere sufficiente calcolare la [differenziazione](https://en.wikipedia.org/wiki/Stationary_process#Differencing) prima che dovrebbe mitigare se non eliminare l'effetto trend.
<br>
Per avere conferma dell'efficacia della differenziazione ripeteremo i test sulla serie differenziata.

In [ ]:
stat = airpassengers["Passengers"].diff()

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(stat)
plt.title("Airpassengers senza effetto trend")

Come possiamo vedere dal plot la serie ora si muove intorno allo zero.

In [ ]:
adf_stat = adfuller(stat.values[1:])
print("p-value: ", round(adf_stat[1], 2))
hypothesis(p_value=round(adf_stat[1], 2), alpha=0.05)

In [ ]:
kpss_stat = kpss(stat.values[1:], nlags="legacy")
print("p-value: ", round(kpss_stat[1], 2))
hypothesis(p_value=round(kpss_stat[1], 2), alpha=0.05)

Possiamo affermare che dopo la differenziazione prima la serie **non presenta la radice unitaria**, è stato rimosso l'effetto trend.
<br>
**N.B. avrete notato che arrotondo il p-value a due cifre, questo per non essere portato a rigettare H0 per uno scarto minimo. Potrebbe essere un errore.**